In [4]:
%load_ext autoreload
%autoreload 2
import numpy as np
from pydantic.dataclasses import dataclass, Field
import dotenv
import interlab
from interlab.queries import query_for_json, query_model
import langchain_community.chat_models
from treetrace import TracingNode
from interlab_zoo.persona.personas import Factory
from interlab_zoo.persona.contacts import link
from interlab_zoo.persona.channels import ChatChannel
from interlab_zoo.persona.utils import and_join
import langchain
import xml.etree.ElementTree as ET

dotenv.load_dotenv()

True

In [5]:
from langchain.cache import InMemoryCache
from langchain.globals import set_llm_cache
set_llm_cache(InMemoryCache())


In [7]:
factory = Factory.from_yaml("audience.yaml")

In [4]:
marie = factory.create_persona("Maria-gpt35")
lilly = factory.create_persona("Lilly-gpt35")

/home/ada/.cache/pypoetry/virtualenvs/interlab-cZHazbI_-py3.10/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [7]:
marie.name

'Maria'

In [40]:
marie.public_description()

'Public information about Maria: age 22\nMaria is a mentor\n'

In [54]:
link([marie, lilly])

In [35]:
with TracingNode("root") as root:
    ChatChannel.query_and_send_message(marie, lilly)
root.display()

NameError: name 'marie' is not defined

In [21]:

class Situation:

    def __init__(self, subject, factory, person_keys):
        self.factory = factory
        self.person_keys = person_keys
        self.players = [self.factory.create_persona(key, goal=f"You want to make an agreement on {subject}") for key in person_keys]        
        names = and_join(p.name for p in self.players)
        judge_model = langchain_community.chat_models.ChatOpenAI(model="gpt-4-turbo-preview")
        self.judge = interlab.actor.llm_actor.OneShotLLMActor("Judge", judge_model, "You observe the following communication:")
        self.subject = subject
        self.agreement_result = None

        @dataclass
        class JudgeAction:
            agreement_subject: str | None = Field(
                description=f"What is ${subject} they agreed on? If there is no clear and unanimous agreement it should be null",
            )
        self.judge_action = JudgeAction

        @dataclass
        class Rating:
            rating: float = Field(
                description=f"Agreement rating on scale 1-10: 1 = worse; 10 = best",
            )
        self.rating_cls = Rating


    def agreement_check(self):
        players = self.players
        names = and_join(p.name for p in self.players)
        question = f"Do all sides ({names}) made an agreement on {self.subject}? If yes, what do they agree on?"
        response = self.judge.query(question)
        prompt = (f"Question: {question}\n\nAnswer: {response}\n. Parse answer into JSON:")
        self.agreement_result = query_for_json(self.judge.model, self.judge_action, prompt).agreement_subject
        return bool(self.agreement_result)


    def query_player(self, player, players):
        names = [p.name for p in players if p is not player]
        names.insert(0, "You")        
        prompt = and_join(names) + f" made an aggrement on {self.subject}: {self.agreement_result}. How are you satisfied?"
        players = [self.factory.create_persona(key) for key in self.person_keys]
        link(players)
        rating = player.query(prompt, expected_type=self.rating_cls)
        return rating.rating
    
    def score_simple(self, idx):
        players = [self.factory.create_persona(key) for key in self.person_keys]
        link(players)
        return self.query_player(players[idx], players)        

    def score_with_history(self, idx):
        return self.query_player(self.players[idx], self.players)            
    
    def score_result(self):
        result = []
        for idx, player in enumerate(self.players):
            row = [self.person_keys[idx], self.agreement_result]
            row.append(self.score_simple(idx))
            row.append(self.score_with_history(idx))
            result.append(row)
        return result


def resolve_direct(sitation, channel=ChatChannel, max_steps_per_player=7):
    players = situation.players
    link(players)
    for i in range(max_steps_per_player):
        for j, sender in enumerate(players):
            others = players[:j] + players[j+1:]
            channel.query_and_send_message_to_all(sender, others, [situation.judge])
        if situation.agreement_check():
            return


def resolve_with_mediator(sitation, mediator_key, channel=ChatChannel, max_steps_per_player=7):
    names = and_join(p.name for p in situation.players)
    mediator = situation.factory.create_persona(
        mediator_key,
        public=f"$name is a mediator to arrange arrange an agreement between {names}",
        goal=f"to help {names} find an agreement on {situation.subject}; you will not watch the movie")
    players = situation.players
    link(players + [mediator])
    for i in range(max_steps_per_player):
        for j, sender in enumerate(players):
            others = players[:j] + players[j+1:]
            channel.query_and_send_message(sender, mediator, others + [situation.judge])
        channel.query_and_send_message_to_all(mediator, players, [situation.judge])
        if situation.agreement_check():
            return


with TracingNode("root") as root:
    situation = Situation("a specific movie name they want to watch together", factory, ["Maria-gpt35", "Connor-gpt35"])
    #resolve_direct(situation)
    with TracingNode("resolving"):
        resolve_with_mediator(situation, "Edward-gpt35")
    print(situation.agreement_result)
    with TracingNode("scoring"):
        scoring = situation.score_result()
    df = pd.DataFrame(scoring, columns=["key", "result", "score_basic", "score_wh"])
    print(df)
root.display()

The Grand Budapest Hotel
            key                    result  score_basic  score_wh
0   Maria-gpt35  The Grand Budapest Hotel          9.0       9.0
1  Connor-gpt35  The Grand Budapest Hotel          7.5       9.0


In [18]:
root.display()